In [1]:
# Import libraries
import os
import sys
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set Pandas row limit
pd.set_option('display.max_rows', 500)

# Load the data from the CSV file
price_data = pd.read_csv('./data/CCL_historical_data.csv')

In [2]:
# Add a "Change in price" column
price_data['change_in_price'] = price_data['c'].diff()
price_data.head()

,datetime,o,h,l,c,v,readable_time,date,change_in_price
0,1571299800000,41.87,41.93,41.87,41.93,400,2019-10-17 03:10:00,2019-10-17,NaN
1,1571300100000,41.94,41.94,41.88,41.88,900,2019-10-17 03:15:00,2019-10-17,-0.05
2,1571301300000,41.84,41.88,41.84,41.88,800,2019-10-17 03:35:00,2019-10-17,0.00
3,1571301600000,41.88,41.88,41.88,41.88,100,2019-10-17 03:40:00,2019-10-17,0.00
4,1571305500000,42.26,42.26,42.26,42.26,300,2019-10-17 04:45:00,2019-10-17,0.38


In [3]:
# Calculate 20 EMA (short term)
ema_20 = price_data['c'].transform(lambda x: x.ewm(span = 20).mean())
price_data['EMA_20'] = ema_20

# Calculate 200 EMA (long term)
ema_200 = price_data['c'].transform(lambda x: x.ewm(span = 200).mean())
price_data['EMA_200'] = ema_200

#  Calculate MACD 
ema_26 = price_data['c'].transform(lambda x: x.ewm(span = 26).mean())
ema_12 = price_data['c'].transform(lambda x: x.ewm(span = 12).mean())
macd = ema_12 - ema_26

ema_9_macd = macd.ewm(span = 9).mean()

# Calculate the VWAP
def vol_weighted_avg_price(date_group):
    vwap_col = []
    volume = date_group['v']
    high = date_group['h']
    low = date_group['l']
    
    # Calculate VWAP
    vwap_col = (volume * ((high + low)/2)).cumsum() / volume.cumsum()
    return pd.Series(vwap_col, index = date_group.index)
    
vwap = price_data.groupby('date').apply(vol_weighted_avg_price)
price_data['VWAP_50'] = vwap.reset_index(level=0, drop=True)

# Store the data in the data fram
price_data['MACD'] = macd
price_data['MACD_EMA'] = ema_9_macd
price_data['MACD_Diff'] = macd - ema_9_macd

price_data.tail(100)

,datetime,o,h,l,c,v,readable_time,date,change_in_price,EMA_20,EMA_200,VWAP_50,MACD,MACD_EMA,MACD_Diff
25455,1593704400000,16.2300,16.3700,16.2300,16.3400,324065,2020-07-02 10:40:00,2020-07-02,0.1099,16.393022,16.662613,16.630653,-0.150032,-0.172664,0.022632
25456,1593704700000,16.3400,16.4400,16.3300,16.4250,370673,2020-07-02 10:45:00,2020-07-02,0.0850,16.396068,16.660249,16.624014,-0.129488,-0.164029,0.034541
25457,1593705000000,16.4261,16.4383,16.3200,16.3918,268503,2020-07-02 10:50:00,2020-07-02,-0.0332,16.395661,16.657578,16.619313,-0.114566,-0.154136,0.039571
25458,1593705300000,16.4000,16.4300,16.3200,16.3430,264727,2020-07-02 10:55:00,2020-07-02,-0.0488,16.390646,16.654448,16.614774,-0.105461,-0.144401,0.038940
25459,1593705600000,16.3500,16.3600,16.2500,16.3000,225856,2020-07-02 11:00:00,2020-07-02,-0.0430,16.382013,16.650921,16.609940,-0.100556,-0.135632,0.035076
25460,1593705900000,16.2934,16.3700,16.2900,16.3200,187295,2020-07-02 11:05:00,2020-07-02,0.0200,16.376107,16.647628,16.606364,-0.093972,-0.127300,0.033328
25461,1593706200000,16.3100,16.3350,16.2500,16.2800,135201,2020-07-02 11:10:00,2020-07-02,-0.0400,16.366954,16.643970,16.603496,-0.090934,-0.120027,0.029093
25462,1593706500000,16.2700,16.2700,16.1100,16.1580,427210,2020-07-02 11:15:00,2020-07-02,-0.1220,16.347054,16.639135,16.591893,-0.097249,-0.115471,0.018222
25463,1593706800000,16.1500,16.1700,16.0900,16.1500,359418,2020-07-02 11:20:00,2020-07-02,-0.0080,16.328287,16.634268,16.581241,-0.101727,-0.112723,0.010996
25464,1593707100000,16.1500,16.2400,16.1499,16.2400,234739,2020-07-02 11:25:00,2020-07-02,0.0900,16.319878,16.630345,16.575508,-0.096897,-0.109557,0.012661


In [6]:
# Indicate buy points
def calculate_buy(df):
    buy_indicators = []
    open_price = df['o']
    close = df['c']
    macd_diff = df['MACD_Diff']
    ema_20 = df['EMA_20']
    vwap = df['VWAP_50']
    

    for op, close, macd, ema, vwap in zip(open_price, close, macd_diff, ema_20, vwap):
        
        # Did the candle stick open above EMA
        if (close > ema):
            pass
        else:
            buy_indicators.append('hold')
            continue
    
        # Is the MACD gapping
        if (macd > 0.02):
            pass
        else:
            buy_indicators.append('hold')
            continue
        
        # MACD extended. Likely a reveral is coming
        if (macd > 0.075):
            buy_indicators.append('hold')
            continue
            
        # Is the price near the 75% VWAP? Don't buy
            
        # Is the price near the 50% VWAP? Buy
        if (close > vwap):
            pass
        else:
            buy_indicators.append('hold')
            continue
            
        buy_indicators.append('BUY')
    
    return pd.Series(buy_indicators)

buy_column = calculate_buy(price_data)

price_data['Buy Indicator'] = buy_column

price_data.tail(500)

,datetime,o,h,l,c,v,readable_time,date,change_in_price,EMA_20,EMA_200,VWAP_50,MACD,MACD_EMA,MACD_Diff,Buy Indicator
25055,1593522000000,16.6800,16.6800,16.5800,16.6000,12022,2020-06-30 08:00:00,2020-06-30,-0.0400,16.671674,16.412467,16.691740,-0.024151,-0.006907,-0.017244,hold
25056,1593522300000,16.5800,16.5800,16.4700,16.4800,16090,2020-06-30 08:05:00,2020-06-30,-0.1200,16.653420,16.413139,16.683020,-0.036108,-0.012747,-0.023361,hold
25057,1593522600000,16.4800,16.5000,16.4000,16.4900,30556,2020-06-30 08:10:00,2020-06-30,0.0100,16.637856,16.413903,16.661970,-0.044267,-0.019051,-0.025216,hold
25058,1593522900000,16.5000,16.5400,16.4800,16.5400,11288,2020-06-30 08:15:00,2020-06-30,0.0500,16.628536,16.415158,16.657062,-0.046167,-0.024474,-0.021692,hold
25059,1593523200000,16.4900,16.5900,16.4900,16.5800,5349,2020-06-30 08:20:00,2020-06-30,0.0400,16.623914,16.416798,16.655298,-0.043938,-0.028367,-0.015571,hold
25060,1593523500000,16.5900,16.6400,16.5400,16.6000,16381,2020-06-30 08:25:00,2020-06-30,0.0200,16.621636,16.418621,16.652417,-0.040095,-0.030713,-0.009383,hold
25061,1593523800000,16.5500,16.6150,16.2500,16.4050,961526,2020-06-30 08:30:00,2020-06-30,-0.1950,16.601004,16.418486,16.493760,-0.052184,-0.035007,-0.017177,hold
25062,1593524100000,16.4100,16.6633,16.2727,16.2817,679271,2020-06-30 08:35:00,2020-06-30,-0.1233,16.570594,16.417125,16.485063,-0.070896,-0.042185,-0.028711,hold
25063,1593524400000,16.2800,16.3100,15.9800,16.1201,964204,2020-06-30 08:40:00,2020-06-30,-0.1616,16.527690,16.414169,16.374895,-0.097639,-0.053276,-0.044364,hold
25064,1593524700000,16.1200,16.2400,15.9000,15.9550,637835,2020-06-30 08:45:00,2020-06-30,-0.1651,16.473148,16.409600,16.321085,-0.130650,-0.068750,-0.061899,hold


In [ ]:

# Incidate sell points


# Calculate P/L based on buy/sell points


# Add running total column 